In [15]:
import numpy as np
from SandboxSafety.Simulator.Dynamics import update_complex_state
from SandboxSafety.Utils import load_conf
from SandboxSafety.Modes import Modes


In [16]:
conf = load_conf("forest_kernel")
m = Modes(conf)
resolution = conf.n_dx
phi_range = conf.phi_range
b = 1 / (resolution)
p = conf.phi_range / (conf.n_phi -1)
v = (conf.max_v - conf.min_v) / (conf.nq_velocity - 1)
s = 2 * conf.max_steer / (conf.nq_steer - 1)
time = conf.kernel_time_step

print(f"Limits: b: {b}, p: {p}, v: {v}, s: {s}")
# These Errors are over two when used. This is the full block size, but theoretically the error will be half this.



Limits: b: 0.0125, p: 0.078525, v: 0.4, s: 0.08


In [17]:
b_state = np.array([0, 0, 0, 3.0, -0.2])
mode_action = np.array([0.2, 2])
# action_id = m.get_mode_id(mode_action[1], mode_action[0])

n_bstate = update_complex_state(b_state, mode_action, time)
dx, dy, phi, vel, steer = n_bstate[0], n_bstate[1], n_bstate[2], n_bstate[3], n_bstate[4]
bq = m.get_safe_mode_id(vel, steer)
print(f"bq: {bq}")
print(f"n_bstate: {n_bstate}")

bq: 17
n_bstate: [-0.01365629  0.29780801 -0.04332517  2.35453794  0.15555556]


In [18]:

options = np.array([[b,b,0, 0, 0]
            ,[b,-b,0, 0, 0]
            ,[-b,b,0, 0, 0]
            ,[-b,-b,0, 0, 0]])

for i, opt in enumerate(options):
    state = b_state + options[i]
    new_state = update_complex_state(state, mode_action, time)

    diff = new_state - n_bstate
    print(f"{i} --> State diff: {diff} ")


0 --> State diff: [0.0125 0.0125 0.     0.     0.    ] 
1 --> State diff: [ 0.0125 -0.0125  0.      0.      0.    ] 
2 --> State diff: [-0.0125  0.0125  0.      0.      0.    ] 
3 --> State diff: [-0.0125 -0.0125  0.      0.      0.    ] 


In [19]:

options = np.array([[0,0,p, 0, 0]
               ,[0,0,-p, 0, 0]])

   
for i, opt in enumerate(options):
    state = b_state + options[i]
    new_state = update_complex_state(state, mode_action, time)

    diff = new_state - n_bstate
    print(f"{i} --> State diff: {diff} ")


0 --> State diff: [0.02340343 0.00015356 0.078525   0.         0.        ] 
1 --> State diff: [-0.02331927 -0.00198896 -0.078525    0.          0.        ] 


In [20]:
# v = 0.2
# s = 0.04
# s = 0

options = np.array([[0,0,0, v, s]
        ,[0,0,0, -v, s]
        ,[0,0,0, v, -s]
        ,[0,0,0, -v, -s]])

for i, opt in enumerate(options):
    state = b_state + options[i]
    new_state = update_complex_state(state, mode_action, time)

    diff = new_state - n_bstate
    print(f"{i} --> State diff: {diff} ")
    qid = m.get_mode_id(state[3], state[4])
    print(f"bq: {bq}, Qid: {qid} ")



0 --> State diff: [0.00867175 0.04378812 0.07551832 0.32323983 0.03225118] 
bq: 17, Qid: 28 
1 --> State diff: [ 0.01079224 -0.03409423  0.06937287 -0.16977026  0.03225118] 
bq: 17, Qid: 22 
2 --> State diff: [-0.01723452  0.0420882  -0.09013878  0.32323983 -0.08      ] 
bq: 17, Qid: 27 
3 --> State diff: [-0.00466302 -0.03486601 -0.05842408 -0.16977026 -0.08      ] 
bq: 17, Qid: 20 


In [21]:

options = np.array([[b/2, b/2, p/2, v/2, s/2]])

for i, opt in enumerate(options):
    state = b_state + options[i]
    new_state = update_complex_state(state, mode_action, time)

    diff = new_state - n_bstate
    print(f"{i} --> State diff: {diff} ")
    qid = m.get_mode_id(state[3], state[4])
    print(f"bq: {bq}, Qid: {qid} ")


0 --> State diff: [0.02243344 0.02787626 0.07525279 0.13709443 0.02112483] 
bq: 17, Qid: 27 


In [24]:
# All options possible

opt1 = np.array([[b,b,0, 0, 0]
        ,[b,-b,0, 0, 0]
        ,[-b,b,0, 0, 0]
        ,[-b,-b,0, 0, 0]])

opt2 = np.array([[0,0,p, 0, 0]
               ,[0,0,-p, 0, 0]])

opt3 = np.array([[0,0,0, v, s]
        ,[0,0,0, -v, s]
        ,[0,0,0, v, -s]
        ,[0,0,0, -v, -s]])

file = open('ErrorLog.txt', 'w')

block_state = np.array([b, b, p, v, s])

i = 0
for o1 in opt1:
        for o2 in opt2:
                for o3 in opt3:
                        state = b_state + o1 + o2 + o3
                        new_state = update_complex_state(state, mode_action, time)
                
                        diff = new_state - n_bstate
                        print(f"{i} --> State diff: {diff} ")
                        file.write(f"{i} --> State diff: {diff} -->  bq: {bq}, Qid: {qid}\n")

                        file.write(f"{diff/block_state}\n")

                        qid = m.get_mode_id(state[3], state[4])
                        print(f"bq: {bq}, Qid: {qid} ")
                        i += 1

file.close()

0 --> State diff: [0.04798339 0.0556265  0.15404332 0.32323983 0.03225118] 
bq: 17, Qid: 28 
1 --> State diff: [ 0.04398791 -0.0221822   0.14789787 -0.16977026  0.03225118] 
bq: 17, Qid: 22 
2 --> State diff: [ 0.0220236   0.05596402 -0.01161378  0.32323983 -0.08      ] 
bq: 17, Qid: 27 
3 --> State diff: [ 0.02851974 -0.02173922  0.02010092 -0.16977026 -0.08      ] 
bq: 17, Qid: 20 
4 --> State diff: [-0.00560917  0.05484448 -0.00300668  0.32323983  0.03225118] 
bq: 17, Qid: 28 
5 --> State diff: [ 0.00261422 -0.02263154 -0.00915213 -0.16977026  0.03225118] 
bq: 17, Qid: 22 
6 --> State diff: [-0.03130226  0.0511176  -0.16866378  0.32323983 -0.08      ] 
bq: 17, Qid: 27 
7 --> State diff: [-0.01273288 -0.02461331 -0.13694908 -0.16977026 -0.08      ] 
bq: 17, Qid: 20 
8 --> State diff: [0.04798339 0.0306265  0.15404332 0.32323983 0.03225118] 
bq: 17, Qid: 28 
9 --> State diff: [ 0.04398791 -0.0471822   0.14789787 -0.16977026  0.03225118] 
bq: 17, Qid: 22 
10 --> State diff: [ 0.0220236

The results show that using a lot more modes (37) leads to significantly better results.

There is still an error on the mode that is selected. But I think the first three are taken into account